# Module 3: Advanced Scala
#### Written by Adam Izraelevitz (adamiz@berkeley.edu)

### Prerequesites
**Module 1: Basic Scala**

## Table of Contents
In this tutorial, you will learn some advanced features of Scala which enable you to write more powerful and expressive Chisel generators. We will focus on the following Scala features:

**[Functional Programming](#funcprog)**
1. [Immutable vs. Mutable Data Structures](#data)  
2. [A Simple Example](#example)  
2. [Map](#map)  
2. [Anonymous Functions](#anonfunc)  
2. [Filter](#filter)  
2. [Reduce](#reduce)  
2. [FoldLeft](#foldleft)

**[Objects and Classes](#objclass)**
2. [Companion Objects](#compobj)  
2. [Case Classes](#caseclass)

**[Type System](#types)**
2. [Static Types](#static)  
2. [Common Idioms](#idiom)
2. [Generic Types](#generic)

**[Match/Case Statements](#match)**

**[Implicits](#implicits)**



# Functional Programming<a name="funcprog"></a>

Functional programming enables progammers to operate on immutable datastructures in a concise and powerful way. Before we delve into this style of programming, we need to understand the difference between immutability and mutability.

## Immutable vs Mutable Datastructures<a name="data"></a>

A datastructure is immutable if, after it is created, it cannot be changed. Conversely, mutable datastructures can be changed after creation.

Like array's in C or ArrayBuffer's in Java, Scala has a corresponding mutable List-like datastructure, `ArrayBuffer[String]`. We can use various update and appending operations to change its contents.

In [ ]:
import scala.collection.mutable //Import mutable collections library
val arraybuf = mutable.ArrayBuffer[String]("a", "b", "c")
println(arraybuf)
// Because ArrayBuffer is mutable, you can update or append values to arraybuf
arraybuf(0) = "A"
println(arraybuf)
arraybuf += "d"
println(arraybuf)

Alternatively, we can use an immutable datastructure, `List[String]`. We cannot use update or appending operations, but instead must create new immutable datastructures containing our changes:

In [ ]:
// Because List is immutable, l will always have the 3 elements "a", "b", and "c"
val list = List("a", "b", "c")
println(list)
// We cannot append a new value or update a value
// list(0) = "A" <- Illegal
// list += "d" <- Illegal

// Instead, we must create a new List with our appended or updated value
val lAppended = list ++ List("d") // ++ returns a new immutable List with both input lists appended
println(lAppended)
val lUpdated = List("A") ++ list.tail // tail returns a new List without the first element
println(lUpdated)

Immutable datastructures have the benefits of eliminating side-effects, making your code's behavior more reliable and predictable.Scala style promotes using immutable datastructures whenever possible; the only exceptions are when a mutable datastructure enables more performant or significantly cleaner code.

## A Simple Example<a name="example"></a>

Generally, we use functional programming to cleanly and concisely create new immutable datastructures from other immutable datastructures.

Suppose we are given a sequence of strings, and want to return a new sequence with each string "doubled":
```scala
val sequence = List("a", "b", "c")
println(sequence.mkString(" ")) //"a b c"
...
val newSequence = ...
...
println(newSequence.mkString(" ")) //"aa bb cc"
```

Using a for-loop and a mutable list, we can mimic how one could do this in C or Java:

In [ ]:
val sequence = List("a", "b", "c")
println(sequence.mkString(" "))

val doubleABuf = mutable.ArrayBuffer[String]()
for(letter <- sequence) {
    doubleABuf += letter + letter
}
println(doubleABuf.mkString(" "))

However, using functional programming, we can do the same but much more concisely:

In [ ]:
println(sequence.mkString(" "))

val doubleSeq = sequence.map(s => s + s)
println(doubleSeq.mkString(" "))

What's going on here?? Let's break it down.

## Map<a name="map"></a>
First, let's rewrite the same example to be more less syntactically concise, but more readable to those unfamiliar with functional programming.

In [ ]:
println(sequence.mkString(" "))

def double(s: String): String = s + s
val doubleSeq = sequence.map(double)
println(doubleSeq.mkString(" "))

As you can see, the function `double` takes a `String` and returns a new "doubled" `String`. We then pass this function, as an argument, to another function called `map`. When called, map returns a new `List` containing the results of applying `double` to every element in `sequence`.

Let's try a different function, `triple`, which appends three copies of a string together:

In [ ]:
println(sequence.mkString(" "))

def triple(s: String): String = ???
val tripleSeq = sequence.map(triple)
println(newSequence.mkString(" "))

As you can see, we can easily change the function we pass the `map`, while abstracting away the actual traversing of the sequence.

## Anonymous Functions<a name="anonfunc"></a>
In our earliest example, we had the following syntax.
```scala
s => s + s
```
This is an anonymous function, or a function that has no name.Think of this like an object with a single method. When we call an anonymous function (e.g. passing it arguments), it calls its method on those arguments. In this case, our argument `s` is doubled.

The following example shows us creating an anonymous function, then calling it:

In [ ]:
val anon = {x: Int => x + x} // Scala usually requires anonymous functions to be wrapped in {}
val result = anon(10)
println(result)

You can see that this is very similar to a Scala method declaration, `def`. Generally, we use anonymous functions when they are simple and referenced only once, and use method declarations otherwise.

Now, you should be able to fully understand our functional programming example:

In [ ]:
println(sequence.mkString(" "))

val doubleSeq = sequence.map(s => s + s)
println(doubleSeq.mkString(" "))

Try writing your own anonymous function, so that the example returns a sequence of tripled strings:

In [ ]:
val tripleSeq = sequence.map(s => ???)
println(tripleSeq.mkString(" "))

## Filter<a name="filter"></a>

Like `map`, the function `filter` takes a function as an argument, but this function returns true or false. Thus, calling `filter` will returns a new sequence containing only elements that return true.

The following code prints a new sequence that only contains numbers greater than 5:

In [ ]:
val numbers = Seq(1, 2, 10, 6, 3, 9)
val filtered = numbers.filter(i => i > 5)
println(filtered.mkString(" "))

## Reduce<a name="reduce"></a>

The function `reduce` will iteratively call its input function on two elements, returning a new element, until only one remains.

The following code prints the total sum of all integers in the sequence:

In [ ]:
val sum = numbers.reduce((a, b) => a + b)
println(sum)

## FoldLeft<a name="foldleft"></a>

Reductions have the problem of failing when the input sequence is empty:

In [ ]:
val empty = Seq[Int]()
empty.reduce((a, b) => a + b)

In these instances, we can use the function `foldLeft`. This function takes two arguments, an initial value and another function. This function takes two arguments - one is the value you are collecting, and the second is an element of the sequence, and it must return the new value you are collecting.

As it turns out, we can use `foldLeft` to implement our `map`, `reduce`, and `filter` examples!

In [ ]:
val mapped = sequence.foldLeft(Seq[String]()){
    (seq, element) => seq ++ Seq(element + element)
}
println(mapped.mkString(" "))

val reduced = numbers.foldLeft(0){
    (sum, element) => sum + element
}
println(reduced)

val filtered = numbers.foldLeft(Seq[Int]()){
    (seq, element) => if (element > 5) seq ++ Seq(element) else seq
}
println(filtered.mkString(" "))

As you can see, `foldLeft` is a very powerful construct, and well worth starting at these examples to fully understand them.

Other useful sequence manipulators can be found here: http://www.scala-lang.org/api/current/scala/collection/immutable/List.html

# Objects and Classes<a name="objclass"></a>

## Companion Objects<a name="compobj"></a>
Like Java, Scala has classes you can define and instantiate. However, when you want a static class, there is no direct support in Java.

Scala has a language feature for these singleton classes, called objects. Often they define a method `apply`, which is  special in that it is called by reference the object, followed by an argument list.

In [ ]:
object MyObject {
    def hi: String = "Hello World!"
    def apply(msg: String) = msg
}
println(MyObject.hi)
println(MyObject(msg)) // equivalent to MyObject.apply(msg)

When a class and an object share the same name and defined in the same file, the object is called a **companion object**.

Companion objects are usually used to run code before or after the class constructor. In the example below, we will count the number of instances of Animal. Note that you must call the companion object's `apply` method for `numberOfAnimals` to be incremented.

In [ ]:
object Animal {
    var numberOfAnimals = 0
    def apply(name: String) = {
        numberOfAnimals += 1
        new Animal(name)
    }
}
class Animal(val name: String)

val bunny = new Animal("Hopper") // Calls the class Animal's constructor
val cat = Animal("Whiskers")     // Calls the object Animal's apply method
println(Animal.numberOfAnimals)  // Prints only one animal!

Chisel uses many companion objects, like Module. When you write `Module(new MyModule)`, you are calling the Module companion object, so Chisel can run background code before and after instantiating `MyModule`.

Can call private members.

## Case Classes<a name="caseclass"></a>

...

# Type System<a name="types"></a>

## Static Types<a name="static"></a>

## Common Idioms<a name="idioms"></a>

### Option

### Seq/List

## Generic Types<a name="generic"></a>

# Match/Case Statements<a name="match"></a>

Scala is a strongly typed language, so the types of all objects are known during runtime. To use this type information to dictate control flow, we use **match** statements.

In [ ]:
val sequence = Seq("a", 1, 0.0)
sequence.foreach { x =>
    x match {
        case s: String => println(s"$x is a String")
        case s: Int    => println(s"$x is an Int")
        case s: Double => println(s"$x is a Double")
    }
}

Match statements have some limitations. Because Scala runs on the JVM, and the JVM does not maintain polymorphic types, you cannot match on them at runtime (because they are all erased). Note that the following example always matches the first case statement, because the `[String]`, `[Int]`, and `[Double]` polymorphic types are erased, and the case statements are **actually** matching on just a `Seq`.

In [ ]:
val sequence = Seq(Seq("a"), Seq(1), Seq(0.0))
sequence.foreach { x =>
    x match {
        case s: Seq[String] => println(s"$x is a String")
        case s: Seq[Int]    => println(s"$x is an Int")
        case s: Seq[Double] => println(s"$x is a Double")
    }
}

# Implicits<a name="implicits"></a>